In [ ]:
pip install langchain_huggingface

In [23]:
from model import RagEmbedding, RagLLM
from doc_parse import chunk, read_and_process_excel, logger

In [24]:
import pandas as pd
from langchain_chroma import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter
import chromadb

In [25]:
pdf_files = ["./data/zhidu_employee.pdf", "./data/zhidu_travel.pdf"]
excel_files = ["./data/zhidu_detail.xlsx"]

In [26]:
r_spliter = RecursiveCharacterTextSplitter(
    chunk_size=128,
    chunk_overlap=30,
    separators=["\n\n", 
                "\n", 
                ".", 
                "\uff0e", 
                "\u3002",
                ",",
                "\uff0c",
                "\u3001'"
                ])

In [27]:
doc_data = []
for pdf_file_name in pdf_files:
    res = chunk(pdf_file_name, callback=logger)
    for data in res:
        content = data["content_with_weight"]
        if '<table>' not in content and len(content) > 200:
            doc_data = doc_data + r_spliter.split_text(content)
        else:
            doc_data.append(content)

OCR is running...


OCR finished.
OCR: 1.0119836249941727
preprocess
Layout analysis finished.
layouts: 1.2993906670017168
preprocess
Table analysis finished.
Text merging finished


OCR is running...
OCR finished.
OCR: 0.4921076250029728
preprocess
Layout analysis finished.
layouts: 0.646266249998007
preprocess
Table analysis finished.
Text merging finished


In [28]:
for i in doc_data:
    print(len(i), "="*10, i)

530 ========== <table><caption>病假发放标准：</caption>
<tr><td  >病假时间</td><td  >连续工龄</td><td  >发放标准</td></tr>
<tr><td></td><td  >不满二年</td><td  >60% </td></tr>
<tr><td></td><td  >已满二年不满四年</td><td  >70% </td></tr>
<tr><td  >6 个月以内病假</td><td  >已满四年不满六年</td><td  >80% </td></tr>
<tr><td></td><td  >已满六年不满八年</td><td  >90% </td></tr>
<tr><td></td><td  >八年以上</td><td  >100% </td></tr>
<tr><td></td><td  >不满一年</td><td  >40% </td></tr>
<tr><td  >6 个月以上病假</td><td  >已满一年不满三年</td><td  >50% </td></tr>
<tr><td></td><td  >连续工龄三年以上</td><td  >60% </td></tr>
</table>
58 ========== 教职工考勤管理制度
第一节适用范围
1、本制度包括了考勤、休假、加班等方面的规定。2、本制度适用于学校全体教职员工。
7 ========== 第二节考勤规定
66 ========== 1、学校的工作时间由学校决定并公布。学校内除特殊岗位特别规定外，全体教职员工均应严格执行学校的作息时间表，不迟到、不早退、不中途离校
120 ========== 。工作时间：星期一至星期四7:55-16:55 星期五7:55-16:152、所有教职工实行考勤打卡制度，工作日内，每天需打卡两次，上午上班一次和下午下班一次。3、教职员工因故(特殊情况除外)晚到或早退，应事先履行请假手续，经批准后方可离校
59 ========== 。原则上，员工请假无论时间长短、假期形式，除急诊病假或突发事件外，一律需按照请假流程。请假，需事先在钉钉系统中提交申请
102 ========== 。请假，需事先在钉钉系统中提交申请。有效的请假流程为：（1）员工休假必须事先向部门负责人申请，将工作交接清楚

In [29]:
for excel_file_name in excel_files:
    data = read_and_process_excel(excel_file_name)
    df = pd.DataFrame(data[8:], columns=data[7])
    data_excel = df.drop(columns=df.columns[11:17])
    doc_data.append(data_excel.to_markdown(index=False).replace(' ', ""))

In [30]:
from langchain_core.documents import Document
documents = []

for chunk in doc_data:
    document = Document(
        page_content=chunk,
        metadata={"source": "test"})
    documents.append(document)

In [31]:
# Chroma服务器连接信息
CHROMA_SERVER_HOST = "localhost"
CHROMA_SERVER_PORT = "8000"  # 默认端口，如果你修改过，请使用实际端口
COLLECTION_NAME = "zhidu_db"
chroma_client = chromadb.HttpClient(
            host=CHROMA_SERVER_HOST,
            port=CHROMA_SERVER_PORT
        )
print(f"成功连接到Chroma服务器: http://{CHROMA_SERVER_HOST}:{CHROMA_SERVER_PORT}")

成功连接到Chroma服务器: http://localhost:8000


INFO:backoff:Backing off send_request(...) for 0.5s (requests.exceptions.ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')))
INFO:backoff:Backing off send_request(...) for 0.3s (requests.exceptions.ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')))
INFO:backoff:Backing off send_request(...) for 1.3s (requests.exceptions.ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')))
ERROR:backoff:Giving up send_request(...) after 4 tries (requests.exceptions.ConnectionError: ('Connection aborted.', ConnectionResetError(54, 'Connection reset by peer')))


In [32]:
embedding_cls = RagEmbedding(model_name="BAAI/bge-m3", device="cpu")

模型加载成功，使用设备: cpu


In [33]:
embedding_db = Chroma.from_documents(
    documents=documents,
    embedding=embedding_cls,
    client=chroma_client,
    collection_name=COLLECTION_NAME
)

AttributeError: 'NoneType' object has no attribute 'get'

In [45]:
query = "迟到有什么规定？"

In [ ]:
related_docs = embedding_db.similarity_search(query, k=2)